In [0]:
from pyspark.sql.functions import col, count, when

In [0]:
try:
    dbutils.fs.unmount("/mnt/operations")
except:
    print("Mount point /mnt/operations not found or already unmounted.")

configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "",
    "fs.azure.account.oauth2.client.secret": '',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d79345a5-6202-4e21-a853-6510ccafa499/oauth2/token"
}

dbutils.fs.mount(
    source = "abfss://operations-data@operationslogistics.dfs.core.windows.net",
    mount_point = "/mnt/operations",
    extra_configs = configs
)

/mnt/operations has been unmounted.


True

In [0]:
%fs
ls "/mnt/operations"

path,name,size,modificationTime
dbfs:/mnt/operations/raw-data/,raw-data/,0,1725302278000
dbfs:/mnt/operations/transformed-data/,transformed-data/,0,1725302293000


In [0]:
spark

In [0]:
inventory_supplier_performance = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/operations/raw-data/inventory_supplier_performance.csv")
logistics_shipping = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/operations/raw-data/logistics_shipping.csv")
production_quality_control = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/operations/raw-data/production_quality_control.csv")
workforce_productivity = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/operations/raw-data/workforce_productivity.csv")

In [0]:
inventory_supplier_performance.show(5)

+------------+----------+---------------+----------------------+--------------------+
|   record_id|      date|inventory_level|order_fulfillment_rate|supplier_performance|
+------------+----------+---------------+----------------------+--------------------+
|REC-95822412|2020-08-12|           7370|                 96.95|               67.34|
|REC-13356886|2019-11-18|           2533|                 87.73|               67.34|
|REC-42868828|2022-01-03|           1367|                 93.89|               61.86|
|REC-28728463|2021-10-01|            875|                 85.52|               96.37|
|REC-83197857|2021-07-26|           7729|                 87.94|               61.81|
+------------+----------+---------------+----------------------+--------------------+
only showing top 5 rows



In [0]:
inventory_supplier_performance.printSchema()

root
 |-- record_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- inventory_level: integer (nullable = true)
 |-- order_fulfillment_rate: double (nullable = true)
 |-- supplier_performance: double (nullable = true)



In [0]:
logistics_shipping.show(5)

+------------+----------+-------+-------------------+------------------+-----------------+--------------------+
|   record_id|      date|returns|customer_complaints|shipping_time_days|delivery_accuracy|transportation_costs|
+------------+----------+-------+-------------------+------------------+-----------------+--------------------+
|REC-95822412|2020-08-12|     74|                 23|               3.0|            91.43|              6543.8|
|REC-13356886|2019-11-18|     58|                 41|              1.28|            99.74|             2404.44|
|REC-42868828|2022-01-03|     59|                 13|              5.85|            93.05|             1066.95|
|REC-28728463|2021-10-01|      3|                 28|              6.82|            97.75|             9401.04|
|REC-83197857|2021-07-26|     25|                 24|              2.69|            95.43|             1495.15|
+------------+----------+-------+-------------------+------------------+-----------------+--------------

In [0]:
logistics_shipping.printSchema()

root
 |-- record_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- returns: integer (nullable = true)
 |-- customer_complaints: integer (nullable = true)
 |-- shipping_time_days: double (nullable = true)
 |-- delivery_accuracy: double (nullable = true)
 |-- transportation_costs: double (nullable = true)



In [0]:
production_quality_control.show(5)

+------------+----------+--------------------------+--------------+----------+-----------+
|   record_id|      date|production_line_efficiency|downtime_hours|yield_rate|defect_rate|
+------------+----------+--------------------------+--------------+----------+-----------+
|REC-95822412|2020-08-12|                     93.39|          5.97|     88.92|        0.5|
|REC-13356886|2019-11-18|                     79.13|          5.25|     88.64|       1.46|
|REC-42868828|2022-01-03|                     88.23|          1.71|      81.3|       4.74|
|REC-28728463|2021-10-01|                     77.76|          6.63|     86.23|        2.6|
|REC-83197857|2021-07-26|                     79.76|          3.89|     85.43|       4.14|
+------------+----------+--------------------------+--------------+----------+-----------+
only showing top 5 rows



In [0]:
production_quality_control.printSchema()

root
 |-- record_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- production_line_efficiency: double (nullable = true)
 |-- downtime_hours: double (nullable = true)
 |-- yield_rate: double (nullable = true)
 |-- defect_rate: double (nullable = true)



In [0]:
workforce_productivity.show(5)

+------------+----------+-----------+---------------------+-----------+------------+
|   record_id|      date|employee_id|employee_productivity|labor_costs|hours_worked|
+------------+----------+-----------+---------------------+-----------+------------+
|REC-95822412|2020-08-12|   EMP-2824|                63.38|   92199.88|       57.54|
|REC-13356886|2019-11-18|   EMP-5506|                65.44|    81838.6|        35.3|
|REC-42868828|2022-01-03|   EMP-4657|               101.05|   64015.25|       24.88|
|REC-28728463|2021-10-01|   EMP-2679|               113.69|    79790.0|       56.87|
|REC-83197857|2021-07-26|   EMP-2424|               108.13|   27455.06|       59.48|
+------------+----------+-----------+---------------------+-----------+------------+
only showing top 5 rows



In [0]:
workforce_productivity.printSchema()

root
 |-- record_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- employee_id: string (nullable = true)
 |-- employee_productivity: double (nullable = true)
 |-- labor_costs: double (nullable = true)
 |-- hours_worked: double (nullable = true)



In [0]:
inventory_supplier_performance.select([count(when(col(c).isNull(), c)).alias(c) for c in inventory_supplier_performance.columns]).show()
logistics_shipping.select([count(when(col(c).isNull(), c)).alias(c) for c in logistics_shipping.columns]).show()
production_quality_control.select([count(when(col(c).isNull(), c)).alias(c) for c in production_quality_control.columns]).show()
workforce_productivity.select([count(when(col(c).isNull(), c)).alias(c) for c in workforce_productivity.columns]).show()

inventory_supplier_performance = inventory_supplier_performance.dropDuplicates()

+---------+----+---------------+----------------------+--------------------+
|record_id|date|inventory_level|order_fulfillment_rate|supplier_performance|
+---------+----+---------------+----------------------+--------------------+
|        0|   0|              0|                     0|                   0|
+---------+----+---------------+----------------------+--------------------+

+---------+----+-------+-------------------+------------------+-----------------+--------------------+
|record_id|date|returns|customer_complaints|shipping_time_days|delivery_accuracy|transportation_costs|
+---------+----+-------+-------------------+------------------+-----------------+--------------------+
|        0|   0|      0|                  0|                 0|                0|                   0|
+---------+----+-------+-------------------+------------------+-----------------+--------------------+

+---------+----+--------------------------+--------------+----------+-----------+
|record_id|date|

In [0]:
inventory_supplier_performance = inventory_supplier_performance.withColumn(
    "inventory_to_fulfillment_ratio", 
    col("inventory_level") / col("order_fulfillment_rate")
)

workforce_productivity = workforce_productivity.withColumn(
    "performance_index", 
    (col("employee_productivity") - col("labor_costs") / col("hours_worked"))
)


In [0]:
inventory_supplier_performance.show(5)
workforce_productivity.show(5)

+------------+----------+---------------+----------------------+--------------------+------------------------------+
|   record_id|      date|inventory_level|order_fulfillment_rate|supplier_performance|inventory_to_fulfillment_ratio|
+------------+----------+---------------+----------------------+--------------------+------------------------------+
|REC-80138498|2022-06-17|           5895|                  86.3|               69.47|              68.3082271147161|
|REC-92533595|2021-02-25|           8500|                 90.64|               95.88|             93.77758164165931|
|REC-95110541|2023-03-29|           8796|                 97.71|               94.48|             90.02149217070925|
|REC-41426254|2021-01-28|           4657|                  91.4|                74.6|             50.95185995623632|
|REC-55876720|2022-09-27|           2154|                 92.23|                97.7|             23.35465683617044|
+------------+----------+---------------+----------------------+

In [0]:
inventory_supplier_performance.write.format("csv").option("header", "true").mode("overwrite").save("/mnt/operations/transformed-data/inventory_supplier_performance_transformed")
logistics_shipping.write.format("csv").option("header", "true").mode("overwrite").save("/mnt/operations/transformed-data/logistics_shipping_transformed")
production_quality_control.write.format("csv").option("header", "true").mode("overwrite").save("/mnt/operations/transformed-data/production_quality_control_transformed")
workforce_productivity.write.format("csv").option("header", "true").mode("overwrite").save("/mnt/operations/transformed-data/workforce_productivity_transformed")